In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 4.6MB/s 
     |████████████████████████████████| 2.3MB 12.3MB/s 
     |████████████████████████████████| 1.2MB 39.7MB/s 
     |████████████████████████████████| 901kB 41.4MB/s 
     |████████████████████████████████| 3.3MB 36.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=dc5de46e3fc5d1564a09a2ef2192076787827ee50e716b1b04842067df5a772c
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [ ]:
!pip install turkishnlp

  Created wheel for turkishnlp: filename=turkishnlp-0.0.61-cp37-none-any.whl size=7557 sha256=68655674544f879795c68958c13782cbe247be5cb9aa4acc5401a0baf13f2faf
  Stored in directory: /root/.cache/pip/wheels/06/11/95/9c04635b9b6cb20e8036e26a6cadaa86b32521a85155b6f1ca
Successfully built turkishnlp


In [ ]:
!pip install --upgrade turkishnlp

Requirement already up-to-date: turkishnlp in /usr/local/lib/python3.7/dist-packages (0.0.61)


In [ ]:
import pandas as pd
import numpy as np
import random
import turkishnlp

In [ ]:
from turkishnlp import detector
obj = detector.TurkishNLP()
obj.download()

Download is successful


In [ ]:
obj.create_word_set()

In [ ]:
text = input()

eğitm dili nedir


In [ ]:
greetings = ["merhaba", "selam", "merhabalar", "selamlar"]

In [ ]:
responses = ["Merhaba", "Selam", "Merhabalar"]

In [ ]:
closings = ["hoşçakal", "görüşürüz", "güle güle", "kendinize iyi bakın"]

In [ ]:
closing_responses = ["Görüşürüz", "Güle güle", "Kendinize iyi bakın"]

In [ ]:
def BilgiBot(input):
  words = obj.list_words(input)
  if words != obj.auto_correct(words):
    corrected_words = " ".join(obj.auto_correct(words))
  else:
    corrected_words = " ".join(obj.auto_correct(words))
  if corrected_words.lower() in greetings:
    print(random.choice(responses) + "," + " " + "Nasıl yardımcı olabilirim?")
  elif corrected_words.lower() == "teşekkürler":
    print("Rica ederim")
  elif corrected_words.lower() == "teşekkür ederim":
    print("Rica ederim")
  elif corrected_words.lower() == "sağol":
    print("Rica ederim")
  elif corrected_words.lower() in closings:
    print(random.choice(closing_responses))
  else:
    getBest(corrected_words,1)

In [ ]:
BilgiBot(text)

 Hukuk Fakültesi,Adalet Meslek Yüksekokulu, Sağlık Bilimleri Meslek Yüksekokulu dışında, İstanbul Bilgi Üniversitesi eğitim dili İngilizcedir.


In [ ]:
df = pd.read_csv("/content/SorularCevaplar.csv")

In [ ]:
df.head()

,Category,Question,Answer
0,Burslar,Öğrenimim süresince hangi burslardan yararlana...,"Üniversiteye girişten itibaren ÖSYM, Sporcu, B..."
1,Burslar,Burs almak için nereye ve ne zaman başvurmam g...,ÖSYM Bursları’ndan yararlanabilmek için üniver...
2,Burslar,Aldığım burs kaç yıl için geçerli?,Burs çeşitlerine göre kullanım süreleri aşağıd...
3,Burslar,Hangi durumlarda bursum kesilir?,ÖSYM bursları not ortalamasına bağlı değildir...
4,Burslar,Üniversite içinde yatay geçiş yaparsam bursum ...,İç yatay geçiş yapmaya hak kazandıysanız ÖSYM...


In [ ]:
df_questions = df.Question

In [ ]:
questions = df_questions.values

In [ ]:
df_answers = df.Answer

In [ ]:
answers = df_answers.values

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

In [ ]:
type(model)

sentence_transformers.SentenceTransformer.SentenceTransformer

In [ ]:
faq_embeddings = model.encode(questions)

In [ ]:
from scipy.spatial.distance import cdist
for q, qe in zip(questions, faq_embeddings):
    distances = cdist([qe], faq_embeddings, "cosine")[0]
    ind = np.argsort(distances, axis=0)[:3]
    print("\n Question: \n "+q)
    for i,(dis,text) in enumerate(zip(distances[ind], [questions[i] for i in ind])):
        print(dis,ind[i],text, sep="\t")


 Question: 
 Öğrenimim süresince hangi burslardan yararlanabilirim?
0.0	0	Öğrenimim süresince hangi burslardan yararlanabilirim?
0.2746000125498731	68	Ders muafiyeti nedir, nasıl ders muafiyeti alabilirim?
0.34325353464582387	132	Bir dersin öğretim elemanına nasıl ulaşabilirim?

 Question: 
 Burs almak için nereye ve ne zaman başvurmam gerekli?
0.0	1	Burs almak için nereye ve ne zaman başvurmam gerekli?
0.21633143988720838	93	Diplomamı ne zaman ve nereden alabilirim?
0.21633144438091068	154	Diplomamı ne zaman ve nereden alabilirim?

 Question: 
 Aldığım burs kaç yıl için geçerli?
0.0	2	Aldığım burs kaç yıl için geçerli?
0.33832164960209943	61	Kaç dönem kayıt dondurabilirim?
0.42331757169651196	75	Bir dönemde en az ve en fazla kaç kredi alabilirim?

 Question: 
 Hangi durumlarda bursum kesilir?
1.1102230246251565e-16	3	Hangi durumlarda bursum kesilir?
0.2747738667811297	159	Aday olmak için hangi şartları taşımalıyım ?
0.30748443918617707	1	Burs almak için nereye ve ne zaman başvurmam g

In [ ]:
def getBest(query, K):
    query_embedding = model.encode([query])
    distances = cdist(query_embedding, faq_embeddings, "cosine")[0]
    ind = np.argsort(distances, axis=0)
    #print("\n"+query)
    for c,i in list(zip(distances[ind],  ind))[:K]:
      if(c>0.3):
        print("Ne dediğinizi anlayamadım lütfen tekrar sorunuz.")

      else:
        #print(c,questions[i], sep="\t") 
        print(answers[i])